In [1]:
import os

In [2]:
%pwd

'/home/izam/coding/dlproject/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/izam/coding/dlproject'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
# creating the entity
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):
        """
        Get data from the config file and convert that into Prepare config type 
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        """
        Returns a PrepareCallbackConfig data type of the configuration of callbacks
        """
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [8]:

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

2023-12-02 08:56:19.995170: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 08:56:19.995212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 08:56:19.997084: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 08:56:20.003871: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 08:56:20.928719: W tensorflow/compiler/tf2

In [9]:
# creating the component
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [10]:
# pipeline

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-12-02 08:56:22,143: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-02 08:56:22,145: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-02 08:56:22,145: INFO: common: created directory at: artifacts]
[2023-12-02 08:56:22,146: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2023-12-02 08:56:22,146: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]


In [11]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-12-02-08-56-22'

In [12]:
prepare_callbacks.config.checkpoint_model_filepath

PosixPath('artifacts/prepare_callbacks/checkpoint_dir/model.h5')